In [ ]:
# %load C:/Users/Saisandeep/Documents/Udacity Data Analyst/Machine-Learning/ud120-projects/tools/email_preprocess.py
#!/usr/bin/python

import pickle
import cPickle
import numpy

from sklearn import cross_validation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif



def preprocess(words_file = "C:/Users/Saisandeep/Documents/Udacity Data Analyst/Machine-Learning/ud120-projects\
               /tools/word_data.pkl", 
               authors_file="C:/Users/Saisandeep/Documents/Udacity Data Analyst/Machine-Learning/ud120-projects\
               /tools/email_authors.pkl"):
    """ 
        this function takes a pre-made list of email texts (by default word_data.pkl)
        and the corresponding authors (by default email_authors.pkl) and performs
        a number of preprocessing steps:
            -- splits into training/testing sets (10% testing)
            -- vectorizes into tfidf matrix
            -- selects/keeps most helpful features

        after this, the feaures and labels are put into numpy arrays, which play nice with sklearn functions

        4 objects are returned:
            -- training/testing features
            -- training/testing labels

    """

    ### the words (features) and authors (labels), already largely preprocessed
    ### this preprocessing will be repeated in the text learning mini-project
    authors_file_handler = open(authors_file, "r")
    authors = pickle.load(authors_file_handler)
    authors_file_handler.close()

    words_file_handler = open(words_file, "r")
    word_data = cPickle.load(words_file_handler)
    words_file_handler.close()

    ### test_size is the percentage of events assigned to the test set
    ### (remainder go into training)
    features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)



    ### text vectorization--go from strings to lists of numbers
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
    features_train_transformed = vectorizer.fit_transform(features_train)
    features_test_transformed  = vectorizer.transform(features_test)

    ### feature selection, because text is super high dimensional and 
    ### can be really computationally chewy as a result
    selector = SelectPercentile(f_classif, percentile=1)
    selector.fit(features_train_transformed, labels_train)
    features_train_transformed = selector.transform(features_train_transformed).toarray()
    features_test_transformed  = selector.transform(features_test_transformed).toarray()

    ### info on the data
    print "no. of Chris training emails:", sum(labels_train)
    print "no. of Sara training emails:", len(labels_train)-sum(labels_train)
    
    return features_train_transformed, features_test_transformed, labels_train, labels_test


In [10]:
# %load C:/Users/Saisandeep/Documents/Udacity Data Analyst/Machine-Learning/ud120-projects/feature_selection/find_signature.py
#!/usr/bin/python

import pickle
import numpy
numpy.random.seed(42)


### The words (features) and authors (labels), already largely processed.
### These files should have been created from the previous (Lesson 10)
### mini-project.
words_file = "C:/Users/Saisandeep/Documents/Udacity Data Analyst/Machine-Learning/ud120-projects/text_learning/your_word_data.pkl" 
authors_file = "C:/Users/Saisandeep/Documents/Udacity Data Analyst/Machine-Learning/ud120-projects/text_learning/your_email_authors.pkl"
word_data = pickle.load( open(words_file, "r"))
authors = pickle.load( open(authors_file, "r") )



### test_size is the percentage of events assigned to the test set (the
### remainder go into training)
### feature matrices changed to dense representations for compatibility with
### classifier functions in versions 0.15.2 and earlier
from sklearn import cross_validation
features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                             stop_words='english')
features_train = vectorizer.fit_transform(features_train)
features_test  = vectorizer.transform(features_test).toarray()


### a classic way to overfit is to use a small number
### of data points and a large number of features;
### train on only 150 events to put ourselves in this regime
features_train = features_train[:150].toarray()
labels_train   = labels_train[:150]



### your code goes here

from sklearn import tree
from time import time

clf = tree.DecisionTreeClassifier()
t0 = time()
clf = clf.fit(features_train, labels_train)
print "training time:", round(time()-t0, 3), "s"

t1 = time()
predicted = clf.predict(features_test)
#print labels_test,predicted

print "predicting time:", round(time()-t1, 3), "s"

accuracy = float(sum(predicted == labels_test))/len(labels_test)
print accuracy



training time: 0.144 s
predicting time: 0.348 s
0.816837315131


In [11]:
for i in range(len(clf.feature_importances_)):
    imp = clf.feature_importances_
    if imp[i]>.1:
        print i, imp[i]

11975 0.105378579003
18849 0.186927243449
21323 0.363636363636


In [12]:
features  = vectorizer.get_feature_names()

In [8]:
features[33614]

u'sshacklmsncom'

In [9]:
features[14343] 

u'cgermannsf'

In [13]:
features[21323]

u'houectect'